In [3]:
import cv2
import dlib
from deepface import DeepFace
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import cv2
import FaceDetectors2
import pygame

FD = FaceDetectors2.FaceDetector()

barba = cv2.imread('faces/barba3.webp', cv2.IMREAD_UNCHANGED)
ceja_izquierda = cv2.imread('faces/ceja-izquierda.png', cv2.IMREAD_UNCHANGED)
ceja_derecha = cv2.imread('faces/ceja-derecha.webp', cv2.IMREAD_UNCHANGED)

barba = cv2.resize(barba,(barba.shape[1] // int(1.5), barba.shape[0]//1))
ceja_izquierda = cv2.resize(ceja_izquierda, (ceja_izquierda.shape[1] // 2, ceja_izquierda.shape[0] // 2))
ceja_derecha = cv2.resize(ceja_derecha, (ceja_derecha.shape[1] // 2, ceja_derecha.shape[0] // 2))



def superponer_filtro(frame, filtro, x, y):
    if x >= 0 and y >= 0 and \
        (x + filtro.shape[1] <= frame.shape[1]) and \
            (y + filtro.shape[0] <= frame.shape[0]):
        
        filtro_resized = cv2.resize(filtro, (int(filtro.shape[1]), int(filtro.shape[0])))
        alpha = filtro_resized[:, :, 3] / 255.0

        for c in range(0, 3):
            frame[y:y + filtro_resized.shape[0], x:x + filtro_resized.shape[1], c] = \
            (
                frame[y:y + filtro_resized.shape[0], x:x + filtro_resized.shape[1], c] * 
                (1 - alpha) + filtro_resized[:, :, c] * alpha
            )

# Conectar a la cámara web
cap = cv2.VideoCapture(0)
pygame.mixer.init()
pygame.mixer.music.load('music/musicaFiltro1.mp3')
pygame.mixer.music.play()
while pygame.mixer.music.get_busy():
    
    pygame.mixer.music.set_volume(0.2)
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.merge([frame, frame, frame])
    
    
    # Detección de rostro y ojos en el fotograma
    values = FD.DetectLargestFaceEyesDNN(frame)
    if values is not None:
        face, eyes, shape = values

        # Dibujar un marco alrededor del rostro
        [x, y, w, h] = face
        if x > -1:
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                superponer_filtro(frame, ceja_izquierda, shape[19][0] - ceja_izquierda.shape[1] // 2-10, shape[19][1] - ceja_izquierda.shape[0] // 2+10)
                superponer_filtro(frame, ceja_derecha, shape[24][0] - ceja_derecha.shape[1] // 2+10, shape[24][1] - ceja_derecha.shape[0] // 2+10)
                superponer_filtro(frame, barba, shape[51][0] - barba.shape[1] // 2, shape[51][1] - barba.shape[0] // 2 + 10)
                

    cv2.imshow('Cam', frame)
    
    # Esc para finalizar
    tec = cv2.waitKey(40)
    if tec == 27:  # Esc
        pygame.mixer.music.stop()
        break

# Cerrar la ventana y liberar la cámara
cap.release()
cv2.destroyAllWindows()


In [15]:
import cv2
import dlib
from imutils import face_utils
import FaceDetectors2
FD = FaceDetectors2.FaceDetector()

majin = cv2.imread('faces/majin.png', cv2.IMREAD_UNCHANGED)
ceja_izquierda = cv2.imread('faces/ceja-izquierda.png', cv2.IMREAD_UNCHANGED)
ceja_derecha = cv2.imread('faces/ceja-derecha.webp', cv2.IMREAD_UNCHANGED)

majin = cv2.resize(majin,(majin.shape[1] // 80, majin.shape[0] // 80))
ceja_izquierda = cv2.resize(ceja_izquierda, (ceja_izquierda.shape[1] // 2, ceja_izquierda.shape[0] // 2))
ceja_derecha = cv2.resize(ceja_derecha, (ceja_derecha.shape[1] // 2, ceja_derecha.shape[0] // 2))


# Cargar el predictor de puntos de referencia faciales
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

# Inicializar el detector de caras de dlib
detector = dlib.get_frontal_face_detector()

def superponer_filtro(frame, filtro, x, y):
    if x >= 0 and y >= 0 and (x + filtro.shape[1] <= frame.shape[1]) and (y + filtro.shape[0] <= frame.shape[0]):
        # Continúa con la superposición

        
        filtro_resized = cv2.resize(filtro, (int(filtro.shape[1]), int(filtro.shape[0])))
        alpha = filtro_resized[:, :, 3] / 255.0

        for c in range(0, 3):
            frame[y:y + filtro_resized.shape[0], x:x + filtro_resized.shape[1], c] = \
            (
                frame[y:y + filtro_resized.shape[0], x:x + filtro_resized.shape[1], c] * 
                (1 - alpha) + filtro_resized[:, :, c] * alpha
            )
# Función para detectar sonrisas
def detectar_sonrisa(shape):
    # Puntos de la boca (48-67)
    boca = shape[48:68]
    # Distancia horizontal entre las comisuras de la boca
    ancho_boca = boca[7][0] - boca[0][0]
    
    # Distancia vertical entre el centro superior e inferior de la boca
    altura_boca = boca[9][1] - boca[3][1]
    # Si la altura es pequeña en comparación con el ancho, se considera una sonrisa
    if altura_boca / ancho_boca < 0.3:  # Ajusta este umbral según sea necesario
        
        return True
    return False

# Capturar video desde la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        if detectar_sonrisa(shape):
            
            values = FD.DetectLargestFaceEyesDNN(frame)
            if values is not None:
                face, eyes, shape = values

                # Dibujar un marco alrededor del rostro
                [x, y, w, h] = face
                if x > -1:
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        superponer_filtro(frame, ceja_izquierda, shape[19][0] - ceja_izquierda.shape[1] // 2-10, shape[19][1] - ceja_izquierda.shape[0] // 2+10)
                        superponer_filtro(frame, ceja_derecha, shape[24][0] - ceja_derecha.shape[1] // 2+10, shape[24][1] - ceja_derecha.shape[0] // 2+10)
                        superponer_filtro(frame, majin, shape[24][0] - majin.shape[1] // 2 -38, shape[24][1] - majin.shape[0] // 2 -20)


    cv2.imshow("Deteccion de Sonrisa", frame)

    # Presionar Esc para salir
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import dlib
import numpy as np
import random
import pygame

# Cargar el detector de rostros de dlib
detector = dlib.get_frontal_face_detector()

# Cargar la imagen de superposición y el bloque
overlay_image = cv2.imread("Minecraft/steve_face.png", -1)  # Imagen que se pone en la cara
# Cargar las imágenes y verificar si fueron cargadas correctamente
diamond_image = cv2.imread("Minecraft/diamond.webp", -1)  # Cargar con transparencia
block_image = cv2.imread("Minecraft/tnt_redimensionada.png", -1)

pygame.mixer.init()
pygame.mixer.music.load("Minecraft/hurt.mp3")

# Parámetros del juego
lives = 3  # Vidas iniciales
block_size = 50  # Tamaño de los bloques
blocks = []  # Lista para almacenar los bloques

# Configuración del bloque de caída
class FallingBlock:
    def __init__(self, x, y, speed, type):
        self.x = x
        self.y = y
        self.speed = speed
        self.type = type

    def move(self):
        self.y += self.speed


# Iniciar la webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a escala de grises (necesario para el detector de dlib)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = detector(gray)

    # Para cada rostro detectado
    for face in faces:
        # Coordenadas del rectángulo de la cara detectada
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        # Verificar si las coordenadas de la cara están dentro de los límites del frame
        if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
            continue  # Si la cara está fuera de la pantalla, no hacer nada y pasar al siguiente rostro

        # Cambiar el tamaño de la imagen de superposición para que coincida con la cara
        overlay_resized = cv2.resize(overlay_image, (w, h))

        # Obtener la región de la cara en el frame
        face_region = frame[y:y+h, x:x+w]

        # Convertir la imagen de superposición a BGRA (si tiene canal alpha)
        if overlay_resized.shape[2] == 4:
            overlay_bgr = overlay_resized[:, :, :3]
            overlay_mask = overlay_resized[:, :, 3]  # Canal alpha
        else:
            overlay_bgr = overlay_resized
            overlay_mask = np.ones(overlay_resized.shape[:2], dtype=np.uint8) * 255

        # Crear una máscara inversa para superponer la imagen
        overlay_inv_mask = cv2.bitwise_not(overlay_mask)

        # Extraer la parte del frame donde colocaremos la imagen
        # Verificar si el bloque está fuera de la pantalla
        bg = cv2.bitwise_and(face_region, face_region, mask=overlay_inv_mask)
            
        # Extraer la parte de la imagen de superposición
        fg = cv2.bitwise_and(overlay_bgr, overlay_bgr, mask=overlay_mask)

        # Sumar ambas imágenes para crear la superposición
        combined = cv2.add(bg, fg)

        # Colocar la imagen combinada en el frame original
        frame[y:y+h, x:x+w] = combined

        # Detectar colisiones con los bloques
        for block in blocks:
            if (block.x < x + w and block.x + block_size > x and
                block.y < y + h and block.y + block_size > y):

                if block.type == "diamond":
                    lives += 1
                    pygame.mixer.music.load("Minecraft/gainlife.mp3")
                else:
                    lives -= 1
                    pygame.mixer.music.load("Minecraft/hurt.mp3")

                pygame.mixer.music.play()
                blocks.remove(block)  # Eliminar el bloque que ha colisionado

    # Generar bloques aleatoriamente
    if random.random() < 0.03:  # Ajustar probabilidad de aparición
        new_block = FallingBlock(random.randint(0, frame.shape[1] - block_size), 0, random.randint(2, 5), random.choice(["diamond", "tnt"]))
        blocks.append(new_block)

        # Mover y dibujar los bloques
    for block in blocks:
        block.move()
        # Eliminar bloques fuera de la pantalla
        if block.y > frame.shape[0]:
            blocks.remove(block)
        else:
            # Cambiar tamaño de la imagen del bloque y dibujar
            if block.type == "diamond":
                block_resized = cv2.resize(diamond_image, (block_size, block_size))
            else:
                block_resized = cv2.resize(block_image, (block_size, block_size))

            if block_resized.shape[2] == 3:  # Si la imagen tiene solo 3 canales (RGB)
                alpha_channel = np.ones((block_resized.shape[0], block_resized.shape[1]), dtype=np.uint8) * 255
                block_resized = cv2.merge([block_resized, alpha_channel])  # Agregar el canal alfa
            
            # Verifica si la imagen tiene 4 canales (BGRA con transparencia)
            if block_resized.shape[2] == 4:
                # Dividir los canales de block_resized (asumimos que es BGRA)
                b, g, r, alpha = cv2.split(block_resized)
                
                # Crear la máscara y su inversa a partir del canal alfa
                mask = alpha
                mask_inv = cv2.bitwise_not(mask)

                # Combinar los canales BGR sin el canal alfa
                block_resized_rgb = cv2.merge((b, g, r))

                # Calcular las coordenadas en el frame donde colocaremos el bloque
                y1, y2 = block.y, block.y + block_size
                x1, x2 = block.x, block.x + block_size

                if y2 <= 0 or y1 >= frame.shape[0] or x2 <= 0 or x1 >= frame.shape[1]:
                    blocks.remove(block)
                    continue  # Saltar al siguiente bloque

                # Ajustar los límites si el bloque se sale de la pantalla
                y1, y2 = max(0, y1), min(frame.shape[0], y2)
                x1, x2 = max(0, x1), min(frame.shape[1], x2)

                # Extraer la región de fondo donde se colocará el bloque
                bg = cv2.bitwise_and(frame[y1:y2, x1:x2], frame[y1:y2, x1:x2], mask=mask_inv[:y2-y1, :x2-x1])
                fg = cv2.bitwise_and(block_resized_rgb[:y2-y1, :x2-x1], block_resized_rgb[:y2-y1, :x2-x1], mask=mask[:y2-y1, :x2-x1])

                # Sumar las imágenes de fondo y primer plano
                combined = cv2.add(bg, fg)

                # Colocar la imagen combinada en el frame original
                frame[y1:y2, x1:x2] = combined
            else:
                print("Error: la imagen del bloque no tiene canal alpha")


    # Dibujar el contador de vidas en la esquina superior izquierda
    cv2.putText(frame, f'Vidas: {lives}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Mostrar el frame resultante
    cv2.imshow("Face Overlay with Falling Blocks", frame)

    # Salir si no hay vidas o si se presiona 'q'
    if lives <= 0 or cv2.waitKey(1) & 0xFF == ord('q'):
        # Crear una pantalla de "Game Over"
        game_over_screen = np.zeros_like(frame)  # Pantalla negra del mismo tamaño que el frame
        cv2.putText(game_over_screen, "Game Over", (int(frame.shape[1] / 4), int(frame.shape[0] / 2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)  # Texto de "Game Over" en rojo

        # Mostrar la pantalla de "Game Over" durante 2 segundos
        for _ in range(60):  # 60 frames (asumiendo aproximadamente 30 FPS)
            cv2.imshow("Face Overlay with Falling Blocks", game_over_screen)
            if cv2.waitKey(30) & 0xFF == ord('q'):  # Permite salir antes si se presiona 'q'
                break

        break  # Salir del bucle principal del juego

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()
